In [4]:
import torch
import torch.nn as nn
import os
import pandas as pd
from data_parsing import process_file

from tqdm import tqdm

print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.13.1+cu117


In [5]:
import os.path as osp

import torch
from torch_geometric.data import Data, Dataset, download_url


class MinCostDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        """If these files are found in the raw directory, download is skipped"""
        return []

    @property
    def processed_file_names(self):
        """If these files are found in the processed directory, processing is skipped"""
        processed_files = []
        path = self.processed_dir
        for file in tqdm(os.listdir(path)):
            file_path = os.path.join(path, file)
            if not os.path.isdir(file_path) and not file == "pre_filter.pt" and not file == "pre_transform.pt":
                processed_files.append(file)

        return processed_files

    def download(self):
        pass

    def process(self):
        idx = 32
        path = self.raw_dir
        for file in tqdm(os.listdir(path)):
            print(file)
            file_path = os.path.join(path, file)
            if os.path.isdir(file_path):
                continue
            # Read data from `raw_path`.
            output = process_file(file_path, 'nx')
            if output["converged"]:
                x = output["x"].type(torch.FloatTensor)
                edge_index = output["edge_index"]
                edge_attr = output["edge_attr"].type(torch.FloatTensor)
                y = output["y"]
                data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, filename = file_path)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [6]:
dataset = MinCostDataset(root = "./data/")
dataset.process()

 51%|█████     | 51/100 [00:00<00:00, 257.38it/s]

netgen_94.txt
netgen_97.txt
netgen_57.txt
netgen_42.txt
netgen_43.txt
netgen_41.txt
netgen_65.txt
netgen_19.txt
netgen_3.txt
netgen_21.txt
netgen_99.txt
netgen_32.txt
netgen_40.txt
netgen_45.txt
netgen_4.txt
netgen_11.txt
netgen_96.txt
netgen_54.txt
netgen_88.txt
netgen_34.txt
netgen_80.txt
netgen_59.txt
netgen_27.txt
netgen_15.txt
netgen_98.txt
netgen_52.txt
netgen_76.txt
netgen_33.txt
netgen_20.txt
netgen_7.txt
netgen_77.txt
netgen_81.txt
netgen_91.txt
netgen_95.txt
netgen_2.txt
netgen_61.txt
netgen_35.txt
netgen_6.txt
netgen_8.txt
netgen_62.txt
netgen_24.txt
netgen_26.txt
netgen_16.txt
netgen_69.txt
netgen_90.txt
netgen_5.txt
netgen_66.txt
netgen_38.txt
netgen_12.txt
netgen_58.txt
netgen_36.txt
netgen_1.txt
netgen_53.txt
netgen_22.txt
netgen_63.txt


100%|██████████| 100/100 [00:00<00:00, 276.51it/s]

netgen_9.txt
netgen_71.txt
netgen_93.txt
netgen_30.txt
netgen_14.txt
netgen_28.txt
netgen_68.txt
netgen_92.txt
netgen_49.txt
netgen_13.txt
netgen_23.txt
netgen_74.txt
netgen_31.txt
netgen_84.txt
netgen_83.txt
netgen_86.txt
netgen_64.txt
netgen_73.txt
netgen_0.txt
netgen_50.txt
netgen_85.txt
netgen_47.txt
netgen_60.txt
netgen_70.txt
netgen_29.txt
netgen_55.txt
netgen_17.txt
netgen_72.txt
netgen_82.txt
netgen_18.txt
netgen_75.txt
netgen_10.txt
netgen_79.txt
netgen_37.txt
netgen_39.txt
netgen_46.txt
netgen_89.txt
netgen_51.txt
netgen_67.txt
netgen_56.txt
netgen_44.txt
netgen_25.txt
netgen_48.txt
netgen_78.txt
netgen_87.txt


In [14]:
def dataset_information(dataset):
    print(dataset)
    print(f"num classes: {dataset.num_classes}")
    print(f"num features: {dataset.num_features}")
    print(f"first graph: {dataset[0]}")

dataset_information(dataset)

100%|██████████| 134/134 [00:00<00:00, 60040.25it/s]


MinCostDataset(132)


100%|██████████| 134/134 [00:00<00:00, 61980.23it/s]


num classes: 128


100%|██████████| 134/134 [00:00<00:00, 61277.45it/s]


num features: 1


100%|██████████| 134/134 [00:00<00:00, 55324.02it/s]


first graph: Data(x=[9559, 1], edge_index=[2, 29682], edge_attr=[29682, 2], y=[1, 1], filename='data/raw/road_flow_01_DC_a.txt')


In [54]:
from torch_geometric.nn import NNConv
import torch.nn.functional as F
import numpy as np

class CBN(torch.nn.Module):
    #TODO cite the colab
    def __init__(self, input_dim, output_dim, edge_feature_dim, args):
        super(CBN, self).__init__()

        hidden_dim = args.hidden_dim
        num_layers = args.num_layers
        dropout = args.dropout

        if num_layers > 1:
            conv_modules = [NNConv(input_dim, hidden_dim, nn.Linear(edge_feature_dim, input_dim * hidden_dim))]
            conv_modules.extend([NNConv(hidden_dim, hidden_dim, nn.Linear(edge_feature_dim, hidden_dim * hidden_dim)) for _ in range(num_layers - 2)])
            conv_modules.append(NNConv(hidden_dim, output_dim, nn.Linear(edge_feature_dim, hidden_dim * output_dim)))

            self.convs = nn.ModuleList(conv_modules)
        else:
            self.convs = nn.ModuleList([NNConv(input_dim, output_dim, nn.Linear(edge_feature_dim, input_dim * output_dim))])

        self.bns = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers - 1)])

        self.post_mp = nn.Linear(hidden_dim, hidden_dim)

        self.num_layers = num_layers

        # Probability of an element getting zeroed
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()
        # self.post_mp.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index, edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, self.training)
        x = self.convs[-1](x, edge_index, edge_attr)
        #x = F.relu(x)
        #x = self.post_mp(x)

        return x

    def dual_value(N, p):
        return np.sum([p[i] * N.b[i] for i in N.V]) + np.sum([N.u[e] * max(0, p[e[1]] - p[e[0]] - N.c[e]) for e in N.E])

    # def loss(self, pred, label, x, edge_index, edge_attr):
    #     # edge_attr[0] is capacity, edge_attr[1] is cost
    #     print(pred.shape)
    #     print(edge_index[0].shape)
    #     print(pred[edge_index[1]].shape)
    #     print(edge_attr[:, 1].shape)
    #     reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
    #     print(reduced_cost.shape)
    #     return label - torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(reduced_cost))

In [26]:
import cvxpy as cp
graph = dataset[50]
N = 20
b = graph.x.numpy().flatten()
costs = graph.edge_attr[:, 1].numpy().flatten()
caps = graph.edge_attr[:, 0].numpy().flatten()
edges = list(zip(*graph.edge_index.numpy()))

y = cp.Variable(N)

obj = -cp.sum([y[i] * b[i] for i in range(N)]) - cp.sum([caps[i]*cp.maximum(0, -costs[i] - y[edges[i][0]] + y[edges[i][1]]) for i in range(len(edges))])
prob = cp.Problem(
    cp.Maximize(obj),
    [y>=0]
  )
prob.solve()
print(y.value, prob.value)

100%|██████████| 134/134 [00:00<00:00, 37705.40it/s]


[ 1.73277154  1.01464668  1.61817639  8.01464668  4.01464668  9.01464668
  3.87053766  9.87818328  4.59986794  5.01464668  5.01464668  6.01464668
  8.01464668 10.01464668  6.1743807   7.18194869  2.01464668  9.01464668
  6.01464668 16.01464668] 854.999999996946


In [23]:
graph.y

tensor([[855]])

In [9]:
edges

[(0, 4),
 (0, 8),
 (0, 9),
 (0, 17),
 (0, 6),
 (3, 8),
 (3, 19),
 (4, 3),
 (4, 18),
 (4, 17),
 (4, 10),
 (4, 13),
 (6, 14),
 (6, 5),
 (8, 6),
 (8, 5),
 (8, 13),
 (14, 17),
 (14, 9),
 (1, 10),
 (1, 16),
 (1, 17),
 (1, 5),
 (1, 4),
 (1, 9),
 (5, 19),
 (5, 15),
 (5, 10),
 (5, 6),
 (5, 17),
 (7, 5),
 (7, 16),
 (7, 19),
 (7, 13),
 (7, 14),
 (7, 11),
 (9, 18),
 (9, 7),
 (9, 4),
 (9, 12),
 (10, 12),
 (10, 8),
 (10, 18),
 (10, 14),
 (10, 5),
 (12, 9),
 (12, 10),
 (12, 7),
 (12, 17),
 (2, 11),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 10),
 (11, 13),
 (11, 5),
 (11, 9),
 (11, 15),
 (13, 15),
 (13, 19),
 (13, 11),
 (13, 5),
 (15, 16),
 (15, 10),
 (16, 17),
 (16, 9)]

In [18]:
graph.edge_attr[:, 1]

tensor([ 1.,  4., 10., 10.,  8.,  3.,  8.,  4.,  4.,  5.,  1.,  6.,  5.,  9.,
         1.,  9., 10.,  6.,  2.,  5.,  1.,  8.,  4.,  3.,  1.,  7.,  5.,  4.,
         2.,  2.,  1.,  1.,  7.,  6.,  4.,  5.,  1.,  7.,  6.,  3.,  8., 10.,
         5.,  5.,  2.,  4., 10.,  4.,  1.,  4.,  9.,  5.,  9.,  4.,  5.,  3.,
         9., 10.,  5.,  1.,  2.,  1.,  4.,  4.,  8.,  3.])

In [11]:
from data_parsing import parse
nodes, edges = parse('./data/raw/netgen_88.txt')

In [12]:

index = {node: index for node, index in zip(nodes, range(len(nodes)))}
x = torch.tensor([supply for supply in nodes.values()]).reshape((-1, 1))
edge_index = torch.reshape(torch.tensor([[index[e[0]], index[e[1]]] for e in edges]), (2, -1))
edge_attr = torch.tensor([list(attributes) for attributes in edges.values()])


In [13]:
edge_index

tensor([[ 0,  4,  0,  8,  0,  9,  0, 17,  0,  6,  3,  8,  3, 19,  4,  3,  4, 18,
          4, 17,  4, 10,  4, 13,  6, 14,  6,  5,  8,  6,  8,  5,  8, 13, 14, 17,
         14,  9,  1, 10,  1, 16,  1, 17,  1,  5,  1,  4,  1,  9,  5, 19,  5, 15,
          5, 10,  5,  6,  5, 17,  7,  5,  7, 16,  7, 19],
        [ 7, 13,  7, 14,  7, 11,  9, 18,  9,  7,  9,  4,  9, 12, 10, 12, 10,  8,
         10, 18, 10, 14, 10,  5, 12,  9, 12, 10, 12,  7, 12, 17,  2, 11,  2,  3,
          2,  4,  2,  5,  2, 10, 11, 13, 11,  5, 11,  9, 11, 15, 13, 15, 13, 19,
         13, 11, 13,  5, 15, 16, 15, 10, 16, 17, 16,  9]])

In [14]:
edge_attr

tensor([[23,  1],
        [45,  4],
        [29, 10],
        [31, 10],
        [45,  8],
        [23,  3],
        [41,  8],
        [23,  4],
        [23,  4],
        [ 3,  5],
        [41,  1],
        [37,  6],
        [23,  5],
        [50,  9],
        [23,  1],
        [17,  9],
        [30, 10],
        [23,  6],
        [14,  2],
        [56,  5],
        [17,  1],
        [24,  8],
        [ 6,  4],
        [40,  3],
        [24,  1],
        [56,  7],
        [47,  5],
        [44,  4],
        [16,  2],
        [38,  2],
        [56,  1],
        [ 8,  1],
        [19,  7],
        [17,  6],
        [21,  4],
        [ 8,  5],
        [56,  1],
        [56,  7],
        [13,  6],
        [43,  3],
        [56,  8],
        [40, 10],
        [18,  5],
        [ 5,  5],
        [ 4,  2],
        [56,  4],
        [ 9, 10],
        [15,  4],
        [44,  1],
        [21,  4],
        [45,  9],
        [21,  5],
        [ 2,  9],
        [12,  4],
        [21,  5],
        [5

In [61]:
class DualLoss(nn.Module):
    def __init__(self):
        super(DualLoss, self).__init__()

    def forward(self, pred, label, x, edge_index, edge_attr):
        # edge_attr[0] is capacity, edge_attr[1] is cost
        #TODO is negative for the moment but if you switch the sign before the second dot product it's always positive ._.
        reduced_cost = pred[edge_index[1]].squeeze() - pred[edge_index[0]].squeeze() - edge_attr[:, 1]
        print(label, -torch.dot(pred.squeeze(), x.squeeze()) - torch.dot(edge_attr[:, 0], F.relu(reduced_cost)))
        return (label + torch.dot(pred.squeeze(), x.squeeze()) + torch.dot(edge_attr[:, 0], F.relu(reduced_cost)))/label

In [33]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [34]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [62]:
args = {
    'num_layers': 5, 
    'batch_size': 32, 
    'hidden_dim': 64, 
    'dropout': 0, 
    'epochs': 500, 
    'opt': 'adam', 
    'opt_scheduler': 'none', 
    'opt_restart': 0, 
    'weight_decay': 5e-3, 
    'lr': 0.00001
}
args = objectview(args)
model = CBN(1, 1, 2, args)
loss_fn = DualLoss()
data = dataset[50]


scheduler, opt = build_optimizer(args, model.parameters())
for i in range(7500):
    model.train()
    opt.zero_grad()
    pred = model(data.x, data.edge_index, data.edge_attr)
    loss = loss_fn(pred, data.y, data.x, data.edge_index, data.edge_attr)
    print(f"loss: {loss.item()}")
    loss.backward()
    opt.step()
print(pred)

100%|██████████| 134/134 [00:00<00:00, 54455.65it/s]


tensor([[855]]) tensor(-190079.4062, grad_fn=<SubBackward0>)
loss: 223.31509399414062
tensor([[855]]) tensor(-189395.0625, grad_fn=<SubBackward0>)
loss: 222.5146942138672
tensor([[855]]) tensor(-188709.4375, grad_fn=<SubBackward0>)
loss: 221.71279907226562
tensor([[855]]) tensor(-188023.0625, grad_fn=<SubBackward0>)
loss: 220.91001892089844
tensor([[855]]) tensor(-187346.6719, grad_fn=<SubBackward0>)
loss: 220.11891174316406
tensor([[855]]) tensor(-186674.1094, grad_fn=<SubBackward0>)
loss: 219.33229064941406
tensor([[855]]) tensor(-186001.0938, grad_fn=<SubBackward0>)
loss: 218.54513549804688
tensor([[855]]) tensor(-185327.3281, grad_fn=<SubBackward0>)
loss: 217.75711059570312
tensor([[855]]) tensor(-184652.7188, grad_fn=<SubBackward0>)
loss: 216.9680938720703
tensor([[855]]) tensor(-183980.0312, grad_fn=<SubBackward0>)
loss: 216.1813201904297
tensor([[855]]) tensor(-183308.4375, grad_fn=<SubBackward0>)
loss: 215.3958282470703
tensor([[855]]) tensor(-182641.1562, grad_fn=<SubBackward0

In [91]:
p = pred.detach().numpy().flatten()
p

array([482.83713, 468.93777,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     , 164.39926,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ], dtype=float32)

In [ ]:
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args):

    print("Node task. test set size:", np.sum(dataset[0]['test_mask'].numpy()))
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    output_dim = 1 # we predict scalar potential values for each vertex
    model = CBN(dataset.num_node_features, output_dim, dataset.num_edge_features, args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        for batch in loader:
            print(f"BATCH {batch}")
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            print(f"BATCH y: {batch.y.shape}")
            # pred = pred[batch.train_mask]
            # label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
          test_acc = test(test_loader, model)
          test_accs.append(test_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])

    return test_accs, losses, best_model, best_acc, test_loader

def test(loader, test_model, is_validation=False, save_model_preds=False, model_type=None):
    test_model.eval()

    correct = 0
    # Note that Cora is only one graph!
    for data in loader:
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            pred = test_model(data).max(dim=1)[1]
            label = data.y

        mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = label[mask]

        if save_model_preds:
          print ("Saving Model Predictions for Model Type", model_type)

          data = {}
          data['pred'] = pred.view(-1).cpu().detach().numpy()
          data['label'] = label.view(-1).cpu().detach().numpy()

          df = pd.DataFrame(data=data)
          # Save locally as csv
          df.to_csv('MinCostFlow-' + model_type + '.csv', sep=',', index=False)

        correct += pred.eq(label).sum().item()

    total = 0
    for data in loader.dataset:
        total += torch.sum(data.val_mask if is_validation else data.test_mask).item()

    return correct / total

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d